# First Steps

This notebook guides you through the first steps using Jupyter notebooks with Exasol.

The notebook demonstrates connecting to an Exasol database instance and using some of its features. 

## 1. Open Secure Configuration Storage

First we need to open the Secure Configuration Storage (SCS) containing the connection information such as the database host, user, password, etc.

In [ ]:
%run utils/access_store_ui.ipynb
display(get_access_store_ui())

## 2. Activate JupySQL

First we will activate the [JupySQL](https://jupysql.ploomber.io) magics:

In [ ]:
%run utils/jupysql_init.ipynb

### 2.1 Create a Database Table

We will use JupySQL to create a database table but other sections will use the table, too:

In [ ]:
%%sql --save udf_output
CREATE OR REPLACE TABLE "{{ai_lab_config.db_schema}}"."SALES" (
    sales_id DECIMAL(18,0),
    position_id INTEGER,
    article_id INTEGER,
    amount INTEGER,
    price DOUBLE,
    voucher_id INTEGER,
    comment VARCHAR(2000),
    canceled BOOL    
)

### 2.2 Importing a CSV File From Remote

This section demonstrates how to import a CSV file from a remote source into the database.

In [ ]:
%%sql --save udf_output
import INTO "{{ai_lab_config.db_schema}}"."SALES"
    FROM CSV AT 'https://dut5tonqye28.cloudfront.net/ai_lab/first_steps/' FILE 'remote.csv'
    COLUMN separator = ';';

### 2.3 Display the Imported Data 

Let's query table `SALES` to display the imported data:

In [ ]:
%%sql --save udf_output
SELECT * FROM "{{ai_lab_config.db_schema}}"."SALES"

## 3. PyExasol

### 3.1 Importing a CSV File From the Local Filesystem

This section demonstrates how to import a CSV file from the local file system into the database.

Function `open_pyexasol_connection()` opens a connection based on the Secure Configuration Storage (SCS).

In [ ]:
from pathlib import Path

with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    path = Path("local.csv")
    import_params = {"column_separator": ";"}
    conn.import_from_file(path, (ai_lab_config.db_schema, "SALES"), import_params)

### 3.2 Importing a CSV File From Remote

This section demonstrates how to import a CSV file from a remote source into the database.

This does not work as import from local is only possible when using a JDBC driver.

```
%%sql 
import INTO "{{ai_lab_config.db_schema}}"."SALES"
FROM LOCAL csv file 'sample.csv' COLUMN separator = ';'
```

## 4. SQL Alchemy

## 5. Using the Exasol Bucket File System

The [Exasol Bucket File System](https://docs.exasol.com/db/latest/database_concepts/bucketfs/bucketfs.htm) (BucketFS) is a powerful feature to exchange non-relational data with the database nodes in an Exasol cluster.

Such data can be arbitrary files including 
* Data to be processed by [User Defined Scripts](https://docs.exasol.com/db/latest/database_concepts/udf_scripts.htm) (UDFs)
* [Script-Language Containers](https://github.com/exasol/script-languages-release) (SLCs)
* Pretrained Large Language AI Models

### 5.1 Uploading a File to the BucketFS

First we will create a sample file:

In [ ]:
%%writefile first_steps/text_file.txt
Hello World!

And now, let's upload the file into the BucketFS.

Function `open_bucketfs_location()` returns a cursor into Exasols BucketFS.

In [ ]:
from exasol.nb_connector.connections import open_bucketfs_location
file = Path("first_steps/text_file.txt")
bfs = open_bucketfs_location(ai_lab_config)
remote = bfs / file.name
remote.write(file.read_bytes())

### 5.2 List the Files in the BucketFS 

We can also list all the files currently available in the BucketFS:

In [ ]:
bfs = open_bucketfs_location(ai_lab_config)
for p in bfs.iterdir():
    print(f'- {p.name}')

### 5.3 Reading a File in the BucketFS

We can also read the contents of a file in the BucketFS:

In [ ]:
import exasol.bucketfs as bfs
content = bfs.as_string(remote.read())
print(f'The file in the BucketFS contains:\n{content}')